In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
582,2024-08-25,Tailândia Tbl,06:00,Pathum Thani,Chaophraya Thunder,1.65,2.19,152.5,1.81,1.85,0.0,0.00,0.00,UHHuNw9C,0.606061,0.456621,0.552486,0.540541,0.062682,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,69,83,4.71,2.55,0.000,0.000,0.198874,0.015456,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
583,2024-08-25,Tailândia Tbl,02:00,Bangkok Tiger Thunder,T-REX,1.10,6.66,176.5,1.83,1.83,0.0,0.00,0.00,0xQOQL9m,0.909091,0.150150,0.546448,0.546448,0.059241,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,89,1.49,1.45,0.000,0.000,1.013277,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
584,2024-08-25,Venezuela Superliga,19:00,Trotamundos,Cocodrilos,2.43,1.47,175.5,1.85,1.81,0.0,0.00,0.00,z59F90C7,0.411523,0.680272,0.540541,0.552486,0.091795,239.092,160.054409,0.669426,1.8,1.643168,0.912871,502.56,3.156,2.291513,0.726081,6.0,133.438,36.329327,0.272256,1.2,1.643168,1.369306,195.32,1.450,0.631031,0.435194,15.0,72,76,6.98,2.57,185.632,146.730,0.348114,0.015456,NaN,5.98,1.196,1.195652,0.676354,0.9,0.223646,0.02,0.004,117.500000,0.000000,0.0,0.000000
585,2024-08-25,Singapura Nbl,02:00,Tagawa,SBA,1.43,2.70,135.5,1.83,1.83,0.0,0.00,0.00,jX9xSfCl,0.699301,0.370370,0.546448,0.546448,0.069671,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,72,63,1.32,8.80,0.000,0.000,0.434879,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
586,2024-08-25,Singapura Nbl,03:30,Tungsan,Eng Tat Hornets,1.83,1.83,143.5,1.83,1.83,0.0,0.00,0.00,G4hbnpnS,0.546448,0.546448,0.546448,0.546448,0.092896,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,57,72,1.83,1.05,0.000,0.000,0.000000,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
587,2024-08-25,Venezuela Superliga,20:00,Marinos,Gladiadores,3.35,1.26,155.5,1.83,1.83,0.0,0.00,0.00,G0n2X34E,0.298507,0.793651,0.546448,0.546448,0.092158,246.652,240.408876,0.974689,1.8,1.643168,0.912871,212.08,2.750,2.401583,0.873303,12.0,112.452,17.677756,0.157203,2.4,1.341641,0.559017,134.90,1.318,0.197661,0.149971,37.0,88,95,2.41,1.42,110.344,110.508,0.641151,0.000000,NaN,-0.64,-0.128,-18.359375,0.720243,0.6,-0.120243,-1.35,-0.270,-0.962963,0.000000,0.0,0.000000
588,2024-08-25,Singapura Nbl,00:30,Tong Whye,SG Basketball,1.19,4.36,139.5,1.83,1.83,0.0,0.00,0.00,n3gcsXGa,0.840336,0.229358,0.546448,0.546448,0.069694,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,84,69,1.30,3.19,0.000,0.000,0.807758,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
589,2024-08-25,Europa Copa Europeia 3X3,13:15,Lituânia 3x3,Suiça 3x3,1.83,1.83,38.5,1.83,1.83,0.0,0.00,0.00,Uw5Ym9go,0.546448,0.546448,0.546448,0.546448,0.092896,0.0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
611,20:00,República Dominicana Lnb,Metros de Santiago,Reales de la Vega,1.97,Back Home
